In [105]:
# install dependencies
%pip install openai jinja2 ipywidgets d20 python-dotenv

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [106]:
# load dotenv file
import os

from dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY") is None:
    raise Exception("missing OPENAI API KEY...")

In [107]:
# load config file
import tomllib

config_name = "config.toml"
config:dict[str, any] = None
with open(config_name, "rb") as f:
    config = tomllib.load(f)

print(f"{config_name} loaded: v{config["version"]}")

config.toml loaded: v0.0.2


In [108]:
# load config
from jinja2 import BaseLoader, Environment

meta = config.get("meta", None)
if meta.get("memories") is None:
    meta["memories"] = []

current_round = 0
messages:dict[str, str] = []

def next_round():
    global current_round, messages
    messages = [] # reset messages
    current_round += 1
    for msg in config["messages"]:
        role = msg.get("role")
        raw_content = msg.get("content")
        # print(f"<< {role} >>")
        
        template = Environment(loader=BaseLoader()).from_string(raw_content)
        content = template.render(meta)
        # print(content)

        messages.append({"role": role, "content": content})
        # print("------")
    
    for msg in config["rounds"][current_round - 1]["messages"]:
        role = msg.get("role")
        raw_content = msg.get("content")
        # print(f"<< {role} >>")

        template = Environment(loader=BaseLoader()).from_string(raw_content)
        content = template.render(meta)
        # print(content)
       
        messages.append({"role": role, "content": content})
        # print("------")

next_round()

In [109]:
# start interactive chat
import re
import functools

import ipywidgets as widgets
from IPython.display import display

from openai import OpenAI
import d20

def count_user_actions():
    count = 0
    for msg in messages:
        if msg["role"] == "user":
            count += 1
    return count

def on_custom_input(b, input:widgets.Text):
    role = "user"
    content = f"{input.value} (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_action(b, index:str):
    role = "user"
    content = f"I have chosen action {index}. (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def on_skill(b, skill:str, difficulty:str):
    role = "user"
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. (TURN: {count_user_actions()+1})"
    messages.append({"role":role, "content":content})
    print(f"USER ACTION: {content}")
    chat()

def on_next_round(b):
    next_round()
    print(f"{messages[-1]["role"].upper()}:")
    print(messages[-1]["content"])
    parse_message(messages[-1]["content"])

# parse message into key-value pairs
pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
action_pattern = re.compile(r"(\d+). *(.+)")
def parse_message(content):
    parsed: dict = {}
    current_key = None
    lines = content.splitlines()
    
    # parse the content line by line
    for line in lines:
        res = pattern.search(line)
        if res is not None:
            current_key = res.group(1)
            if res.group(2) is not None:
                parsed[current_key] = res.group(2)
            else:
                parsed[current_key] = ""
        elif current_key is not None:
            parsed[current_key] += "\n" + line
    
    keys = parsed.keys()
    # possible actions
    vbox = widgets.HBox()
    if "possible actions" in keys:
        ma = action_pattern.finditer(parsed["possible actions"])
        for m in ma:
            action_index=m.group(1)
            action_button = widgets.Button(description=action_index)
            action_button.on_click(functools.partial(on_action, index=action_index))
            vbox.children += (action_button,)
        custom_action_button = widgets.Button(description="CUSTOM")
        custom_action_button.on_click(functools.partial(on_custom_action))
        vbox.children += (custom_action_button,)
        display(vbox)
    
    # skill check
    if "skill" in keys and "difficulty" in keys:
        skill = parsed.get("skill")
        difficulty = parsed.get("difficulty")
        skill_button = widgets.Button(description=f"SKILL CHECK - {skill.upper()}", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        display(skill_button)
    
    if "round summary" in keys:
        summary = parsed["round summary"]
        next_round_button = widgets.Button(description="NEXT ROUND")
        meta["memories"].append(summary)
        
        next_round_button.on_click(on_next_round)
        display(next_round_button)

def chat():
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    stream = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )
    print("\nASSISTANT:")
    generated = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            print(delta, end="")
            generated += delta
    messages.append({"role": "assistant", "content": generated})
    parse_message(messages[-1]["content"])

print(f"{messages[-1]["role"].upper()}:")
print(messages[-1]["content"])
# parse the last message
parse_message(messages[-1]["content"])

ASSISTANT:
You are standing in front of the haunted mansion. 
The mansion is old and decrepit, with ivy growing up the walls and the windows boarded up. 
The front door is slightly ajar, inviting you inside. 

What do you do?

possible actions:
1. Enter the mansion.
2. Take some photos of the mansion's exterior.
3. Talk to the butler. (may need a persuasion skill check)


USER ACTION: I have chosen action 3. (TURN: 1)

ASSISTANT:
You approach the mansion's entrance and find the butler waiting by the door, his expression unreadable. He seems wary of strangers but also curious about your intentions.

skill: persuasion
difficulty: medium

Button(description='SKILL CHECK - PERSUASION', layout=Layout(width='auto'), style=ButtonStyle())

USER ACTION: I am making a skill check using PERSUASION against a difficulty of MEDIUM.
And I rolled a 73 for a result of FAILURE. (TURN: 2)

ASSISTANT:
The butler listens to your attempt to engage him in conversation, but he remains unimpressed. He curtly tells you that the master of the house is not receiving visitors and advises you to leave the premises immediately. It's clear that your attempt to persuade him has failed, and he is not willing to offer any assistance or information.

You are still outside the mansion, with the butler making it clear that he will not be of help.

possible actions:
1. Enter the mansion despite the butler's warning.
2. Take some photos of the mansion's exterior.
3. Attempt to sneak around the mansion to find another way in.

USER ACTION: I have chosen action 2. (TURN: 3)

ASSISTANT:
You decide to take a step back and capture the eerie beauty of the mansion with your camera. The mansion, with its overgrown ivy, boarded-up windows, and an aura of mystery, makes for a compelling subject. You take several photos from different angles, making sure to document its exterior thoroughly. This might come in handy later, either for your investigation or to sell the story to a newspaper or magazine.

After finishing with the photography, you realize that the butler has disappeared, leaving the front door even more inviting than before.

possible actions:
1. Enter the mansion.
2. Circle around to the back of the mansion to investigate.
3. Review the photos you just took for any clues.

USER ACTION: I have chosen action 1. (TURN: 4)

ASSISTANT:
You gather your courage and step through the front door of the mansion. The air inside is musty, filled with the scent of old wood and a hint of something unidentifiable. The foyer is dimly lit by the light filtering in through the boarded-up windows, casting eerie shadows across the floor. There's a grand staircase leading to the upper floors, and several doors leading to other rooms on the ground floor.

As you take in your surroundings, you remember the map you were given. It shows several key areas on the ground floor: the library, the dining room, and the living room, among others.

possible actions:
1. Explore the library.
2. Head towards the dining room. (remember the trigger)
3. Investigate the living room.

USER ACTION: I have chosen action 3. (TURN: 5)

ASSISTANT:
You decide to investigate the living room, curious about what you might find there. The door creaks loudly as you push it open, revealing a large room filled with antique furniture covered in dust sheets. The air feels colder here, and a sense of unease settles over you.

As you walk further into the room, you notice a fireplace at the far end, several old portraits hanging on the walls, and a grand piano covered in dust. There's an air of neglect, as if the room hasn't been used in years. You also spot a bookshelf filled with old books and a few scattered newspapers.

Suddenly, as you're about to end your turn in the living room, you hear a faint, eerie sound coming from the direction of the basement. It sounds like a distant whisper or a sigh, chilling you to the bone. It's a reminder of your ultimate goal - to uncover the truth behind the curse of this mansion.

round summary: After attempting to persuade the butler without 

Button(description='NEXT ROUND', style=ButtonStyle())